In [1]:
import pydeck as pdk
import pandas as pd

### 전입 전출자 Mapbox Arc 모양으로 표현

In [2]:
def sido(df):
    return df.split(' ')[0]

In [3]:
geongkibuk=["김포시","고양시","양주시","의정부시","파주시","연천군","동두천시","포천시","구리시","의정부시","남양주시","가평군","양평군"]
geongkinam=["하남시",'광주시',"여주시","이천시","성남시","용인시","안성시","평택시","오산시","수원시","의왕시","과천시","군포시","안양시","광명시","부천시","시흥시","안산시","화성시","평택시"]

#### 각 시도별 위도, 경도 표현

In [4]:
lat=[37.89081,37.27703,37.73458,35.23830,36.57854,35.15441,35.83421,36.34026,35.16788,37.55626,35.54421,37.45603,34.99069,35.82812,33.49802,36.66962,36.62539]
lot=[127.73846,127.00787,127.068296,128.69239,128.77940,126.84307,128.56243,127.39788,129.04583,126.99139,129.24908,126.70575,126.47796,127.11366,126.53174,126.78211,127.50361]

#### 각 시도별 세종시 전입자 수 및 경로 그래프

In [5]:
data_arc=pd.read_csv('./Data/20.세종시_전입자수.csv')
data_arc2=data_arc.drop(['년월','세종전입행정동','전출읍면동'],axis=1)
data_arc2['시찾기']=data_arc2['전출시군구'].apply(sido)
idx_not=data_arc2[data_arc2['시찾기']=='세종특별자치시'].index
data_arc3=data_arc2.drop(idx_not)
data_arc3=data_arc3.reset_index(drop=True)

In [6]:
rororo=[]
for i in range(len(data_arc3)):
    if data_arc3['시찾기'][i] in geongkibuk:
        rororo.append('경기북부')
    elif data_arc3['시찾기'][i] in geongkinam:
        rororo.append('경기남부')
    else:
        rororo.append(data_arc3['전출시'][i])
data_arc3['시도발견']=rororo

In [7]:
data_arc32=data_arc3.groupby('시도발견').sum()
data_arc32['위도']=lat
data_arc32['경도']=lot
#세종특별자치시 : [36.50319, 127.24628]
data_arc32['세종위도']=36.50319
data_arc32['세종경도']=127.24628
data_arc32=data_arc32.reset_index()

In [8]:
layer = pdk.Layer(
    'ArcLayer',
    data_arc32,
    get_source_position=['경도', '위도'],
    get_target_position=['세종경도', '세종위도'],
    get_width='전입자수*0.0001',
    get_source_color=[142, 133, 255],
    get_target_color=[12, 0, 164],
    pickable=True,
    auto_highlight=True
)
view_state = pdk.ViewState(
    latitude=36.50319,
    longitude=127.24628,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)
r = pdk.Deck(layers=[layer], initial_view_state=view_state,map_provider="mapbox",map_style=pdk.map_styles.ROAD)#.SATELLITE)
r.to_html('hexagon-example.html')

#### 각 시도별 세종시 전출자 수 및 경로 그래프

In [10]:
data2_arc=pd.read_csv('./Data/21.세종시_전출자수.csv')
data2_arc2=data2_arc.drop(['년월','세종전출행정동','전입읍면동'],axis=1)
data2_arc2['시찾기']=data2_arc2['전입시군구'].apply(sido)
idx_not=data2_arc2[data2_arc2['시찾기']=='세종특별자치시'].index
data2_arc3=data2_arc2.drop(idx_not)
data2_arc3=data2_arc3.reset_index(drop=True)

In [11]:
rororo2=[]
for i in range(len(data2_arc3)):
    if data2_arc3['시찾기'][i] in geongkibuk:
        rororo2.append('경기북부')
    elif data2_arc3['시찾기'][i] in geongkinam:
        rororo2.append('경기남부')
    else:
        rororo2.append(data2_arc3['전입시'][i])
data2_arc3['시도발견']=rororo2
data2_arc32=data2_arc3.groupby('시도발견').sum()
data2_arc32['위도']=lat
data2_arc32['경도']=lot
data2_arc32['세종위도']=36.50319
data2_arc32['세종경도']=127.24628

In [12]:
layer = pdk.Layer(
    'ArcLayer',
    data2_arc32,
    get_source_position=['경도', '위도'],
    get_target_position=['세종경도', '세종위도'],
    get_width='전출자수*0.0001',#'1 + 0000000.1* 전입자수',
    get_source_color=[255, 109, 109],
    get_target_color=[142, 0, 0],
    pickable=True,
    auto_highlight=True
)
view_state = pdk.ViewState(
    latitude=36.50319,
    longitude=127.24628,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

# Combined all of it and render a viewport
r = pdk.Deck(layers=[layer], initial_view_state=view_state,map_provider="mapbox",map_style=pdk.map_styles.ROAD)
r.to_html('hexagon-example.html')